In [10]:
# !conda install -c conda-forge python-dotenv -y
import numpy as np
import pandas as pd
from deap import base, creator, tools
from functools import reduce
from dotenv import load_dotenv
import finlab
import os
import plotly.express as px
from finlab import data
from finlab.backtest import sim
from finlab.dataframe import FinlabDataFrame
from finlab.portfolio import Portfolio
from finlab.plot import plot_tw_stock_candles
from finlab.data import indicator

# 載入.env檔案中的環境變數

load_dotenv()
# 這裡要替換成自己的FinLab VIP token
token = os.environ.get("FINLAB_TOKEN")
finlab.login(token)



輸入成功!


## K線波動率
finlab 文章:
- V轉指標：台股市場 ATR 波動率指標 : https://www.finlab.tw/tw-stock-market-atr/
- 新手看價，老手看量，高手看波動率 : https://www.finlab.tw/low_volatility_research/

--- 
### K 線波動率的定義與公式
colab link: https://colab.research.google.com/drive/1YHzthYj1-ruAFsHIrtdz5WsHMpBlthaW#scrollTo=fDkSslZlunes  
K線波動率（K-Line volatility）是一種股市波動率的計算方法，常用於股票市場技術分析中。它是基於K線圖樣的變化來計算的，可以用來評估股票市場的波動程度。  
K線圖是用來表示股票市場走勢的一種圖表。每個K線包含了一段時間內的開盤價、最高價、最低價和收盤價。  

K線波動率的計算基於這些價格資訊所計算  
這個指標是由飆股的長相作者所自創  
比起另一個常用的波動指標 ATR ，更能完整涵蓋波動細節。  

在計算K線波動率之前，首先要先計算出「單日股價漲跌幅幅度」。  

- 陽日線算法-紅K(收盤價 >= 開盤價)
波動幅度 = |昨日收盤價 – 當日開盤價| + |開盤價 – 最低價| + |最低價 – 最高價| + |最高價 – 收盤價|

- 陰日線算法-綠k(收盤價 < 開盤價)
波動幅度 = |昨日收盤價 – 當日開盤價| + |開盤價 – 最高價| + |最高價 – 最低價| + |最低價 – 收盤價|

- 飆股的長相 – 自創K線波動率算法
波動率(%) = 「單日波動幅度」的月平均值 / 收盤價的月平均值 * 100  

建議投資風險謹慎的投資人最好選擇波動率4%以下的股票，避開6%以上的高風險股票。  
  
**台股8%以下比較容易出現飆股**
- 因子: condition = volatility <= 8


In [ ]:
def compute_candle_volatility(timeperiod=20):
    with data.universe(market='TSE_OTC'):
        close = data.get('etl:adj_close') # 還原收盤價
        high = data.get("etl:adj_high")
        low = data.get("etl:adj_low")
        open_ =  data.get("etl:adj_open")

    bullish_candle = close >= open_ 
    bullish_volatility = abs(close.shift() - open_) + abs(open_ - low) + abs(low - high) + abs(high - close) # 紅k
    bearish_volatility = abs(close.shift() - open_) + abs(open_ - high) + abs(high-low) + abs(low - close) # 綠k
    candle_volatility = FinlabDataFrame(np.nan, index=close.index, columns=close.columns)
    candle_volatility[bullish_candle] = bullish_volatility
    candle_volatility[~bullish_candle] = bearish_volatility
    volatility = candle_volatility.average(timeperiod) / close.average(timeperiod) * 100
    return volatility
volatility = compute_candle_volatility()


print("k線波動率:",volatility)
condition = volatility > 8
非波動率大於8 = ~condition
波動率大於8 = condition.iloc[-1][condition.iloc[-1]==True].index.tolist()
print("波動率>8:",波動率大於8)

k線波動率: symbol          1101      1102      1103      1104  1107      1108      1109  \
date                                                                           
2007-04-23       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
2007-04-24       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
2007-04-25       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
2007-04-26       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
2007-04-27       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
...              ...       ...       ...       ...   ...       ...       ...   
2025-03-10  3.506570  2.805349  2.597403  2.278866   NaN  1.643192  1.587302   
2025-03-11  3.634628  2.880293  2.677119  2.439024   NaN  1.807834  1.556802   
2025-03-12  3.772206  3.095939  2.667852  2.389937   NaN  1.837621  1.554840   
2025-03-13  3.968196  3.128295  2.658942  2.325776   NaN  1.817575  1.580862   
2025-03-14  3.834658  2.993872  2

### 繪製k線波動率
finlab技術指標:https://doc.finlab.tw/reference/data/#finlab.data.indicator
finlab技術圖:https://doc.finlab.tw/reference/plot/#finlab.plot.plot_tw_stock_candles

In [15]:
overlay_func={
              'ema_5':indicator('EMA',timeperiod=5),
              'ema_10':indicator('EMA',timeperiod=10),
              'ema_20':indicator('EMA',timeperiod=20),
              'ema_60':indicator('EMA',timeperiod=60),
              }
k,d = indicator('STOCH')
macd = indicator('MACD')
rsi = indicator('RSI')
technical_func = [{'volatility':volatility},{'MACD':macd},{'RSI':rsi}]
plot_tw_stock_candles(stock_id='2330',recent_days=200,adjust_price=False,overlay_func=overlay_func,technical_func=technical_func)

In [16]:

# 排除金融股
security = data.get('security_categories')
category_range = [ind for ind in list(set(security['category'])) if '金融' not in ind]


with data.universe(market='TSE_OTC',category=category_range):

    close = data.get("price:收盤價")
    high = data.get("price:最高價")
    low = data.get("price:最低價")
    vol =  data.get("price:成交股數")
    volatility = compute_candle_volatility()
    rev = data.get('monthly_revenue:當月營收')
    
    # 收盤價近5日至少有1日創收盤價近120日創新
    condition1 = (close == close.rolling(120).max()).sustain(5,1)
    # 近60日股價高低區間在30%內
    condition2 = (1 - low.rolling(60).min()/high.rolling(60).max()) < 0.3
    # 收盤價低於整體市場分級的40%
    condition3 = close <= close.quantile_row(0.4)
    # 收盤價低於25元
    condition4 = close <= 25
    # 5日均大於100張
    condition5 = vol.average(5) > 100*1000
    # k線波動率
    condition6 = volatility <= 8
    # 營收短期動能大於長期動能
    condition7 = rev.average(2) > rev.average(12) 

    # 交集所有條件
    position = condition1 & condition2 & condition3 & condition4 & condition5 & condition6 & condition7

    # 最後再挑選前5低價的標地
    position = close * (position.astype(int))
    position = position[position > 0].is_smallest(5)
    
    # 每月底產生訊號、隔月第一個交易日進場、開盤價進出、每檔標的持有部位上限為20%、設定交易手續費折扣
    report = sim(position, resample="M", name="低股價低波動營收成長", upload=True, stop_loss=0.03, trade_at_price='open',position_limit=1/5, fee_ratio=1.425/1000*0.3, mae_mfe_window=40)

